In [2]:
import pandas as pd
import numpy as np

In [3]:
diagnosis_df = pd.read_csv('../raw/trainingSet/training_SyncDiagnosis.csv')
patient_df = pd.read_csv('../raw/trainingSet/training_SyncPatient.csv')
transcript_df = pd.read_csv('../raw/trainingSet/training_SyncTranscript.csv')
conversion_df = pd.read_csv('../raw/trainingSet/training_SyncTranscriptDiagnosis.csv')
medication_df = pd.read_csv('../raw/trainingSet/training_SyncMedication.csv')

patient_df.head()

,PatientGuid,DMIndicator,Gender,YearOfBirth,State,PracticeGuid
0,FB6EFC3D-1A20-4497-9CBD-00027CC5D220,0,M,1929,SD,7BF4DAD8-5F67-4985-B911-20C9E89A3737
1,C6746626-6783-4650-A58F-00065649139A,0,F,1985,TX,E7101967-2FF1-4B0F-8129-B0B429D1D15C
2,E05C6E8F-779F-4594-A388-000C635AE4D3,0,F,1984,NJ,FC01A799-1CAF-464F-A86F-8A666AB86F32
3,EAEBD216-F847-4355-87B2-000D942E08F0,0,M,1959,OH,EEBC95EF-79BE-4542-892E-98D3166BAB20
4,C7F10A80-4934-42D2-8540-000FBEBA75C8,0,F,1990,FL,677BA32E-B4C4-48F2-86E4-08C42B135401


In [4]:
medication_df.head()

,MedicationGuid,PatientGuid,NdcCode,MedicationName,MedicationStrength,Schedule,DiagnosisGuid,UserGuid
0,0D6B339C-AB1F-4FF1-B231-000026C414C9,A7F1591C-9DB8-4F12-A0B8-15AC12C2B801,23490544301,Digoxin oral tablet,125 mcg (0.125 mg),NaN,DA833B28-CE04-4547-A94F-8A4E6C187A3B,1247EA46-61C3-4E91-ACF1-9C54D920ED05
1,72ADD073-CDA0-4F85-B6CB-00008B645E90,7A88D55F-22A9-4126-861D-92F0DA2ED701,143126730,Lisinopril oral tablet,10 mg,NaN,3ED5B90C-22A1-46EE-9CF5-027E684627D8,89049E64-8E9C-4863-8CF2-B6874235FA14
2,D35E505A-02F8-4EFD-A460-00042F85EF3E,F0B80C93-D371-44BE-B464-530C97FE6C02,456069801,Tessalon (benzonatate) oral capsule,200 mg,NaN,0B6C3285-1D8C-4C0F-9928-95EC226CB2FE,7511BCEC-9E46-4808-99A7-5B30778B6E84
3,FFB036B9-A636-48F2-967C-0005F8D78574,FF32CBDA-DDB1-4730-81C5-ACB0F321C2C5,60432060504,Promethazine VC Plain (phenylephrine-promethaz...,5 mg-6.25 mg/5 mL,NaN,DE2E15A5-5080-4141-A061-C81EC739C0A4,7CA1DC13-E602-4296-805F-A64DC8950C5A
4,AB095685-A703-4D81-8678-0009396BF105,1ECD93C8-75BD-4D3C-8D15-E2A3D0373090,247211730,Lexapro (escitalopram) oral tablet,10 mg,NaN,16B189A4-359C-41E1-BAA3-3E081EA24C41,92EBF1FE-3BE4-4367-83DD-C34F58B0B6C6


In [5]:
def dense_dataframe(df, on):
    """dense the dataframe by combining entries of same value of a given column"""
    lst = []
    for index, value in df.iteritems(): # loop through columns in the dataframe
        # combines rows of values of each column to a list, and store the lists into a list
        lst.append(df.groupby(on)[index].apply(list)) 
        
    processed_df = pd.concat(lst, axis=1) # convert the list of lists of values to a dataframe 
    processed_df[on] = processed_df[on].apply(lambda x:x[0]) # retrieve the same value of 'on' column from the list    
    return processed_df

processed_conversion_df = dense_dataframe(conversion_df, 'TranscriptGuid')
processed_transcripts_df = dense_dataframe(transcript_df, 'PatientGuid')
processed_medications_df = dense_dataframe(medication_df, 'PatientGuid')
processed_transcripts_df = processed_transcripts_df.merge(processed_medications_df, on='PatientGuid')

In [6]:
conversion_dict1 = processed_conversion_df.set_index('TranscriptGuid')['DiagnosisGuid'].to_dict()
conversion_dict2 = diagnosis_df.set_index('DiagnosisGuid').to_dict()

In [7]:
def get_diagnoses(transcripts):
    diagnoses, descriptions_all, acute_all, icd9_all, start_all, stop_all = [], [], [], [], [], []

    for i in transcripts:
        if i in conversion_dict1:
            diag = conversion_dict1[i]
            diagnoses.append(diag)

            descriptions_one, acute_one, icd9_one, start_one, stop_one = [], [], [], [], []
            for k in diag:
                descriptions_one.append(conversion_dict2['DiagnosisDescription'][k])
                acute_one.append(       conversion_dict2['Acute'][k])
                icd9_one.append(        conversion_dict2['ICD9Code'][k])
                start_one.append(       conversion_dict2['StartYear'][k])
                stop_one.append(        conversion_dict2['StopYear'][k])
            
            descriptions_all.append(descriptions_one)
            acute_all.append(acute_one)
            icd9_all.append(icd9_one)
            start_all.append(start_one)
            stop_all.append(stop_one)
            
    return diagnoses, descriptions_all, acute_all, icd9_all, start_all, stop_all

def combine_data(conversion, transcripts):
    """ Matches diagnosis data with each transcript """
    
    transcripts['CombinedData'] = transcripts.TranscriptGuid.apply(get_diagnoses)
    
    transcripts['DiagnosisGuid']        = [x[0] for x in transcripts['CombinedData']]
    transcripts['DiagnosisDescription'] = [x[1] for x in transcripts['CombinedData']]
    transcripts['Acute']                = [x[2] for x in transcripts['CombinedData']]
    transcripts['ICD9Code']             = [x[3] for x in transcripts['CombinedData']]
    transcripts['StartYear']            = [x[4] for x in transcripts['CombinedData']]
    transcripts['StopYear']             = [x[5] for x in transcripts['CombinedData']]
        
    return transcripts
        
combined_data = combine_data(processed_conversion_df, processed_transcripts_df)
combined_data.head()

,TranscriptGuid,PatientGuid,VisitYear,Height,Weight,BMI,SystolicBP,DiastolicBP,RespiratoryRate,HeartRate,...,MedicationStrength,Schedule,DiagnosisGuid,UserGuid_y,CombinedData,DiagnosisDescription,Acute,ICD9Code,StartYear,StopYear
0,"[C024B9A8-C201-4013-9DD4-16877BFCBDCF, A04B906...",00023761-9D8D-445B-874C-2424CC7CF620,"[2009, 2009, 2009]","[68.0, 67.0, 67.0]","[182.0, 184.0, 185.0]","[27.67, 28.815, 28.972]","[122.0, 134.0, 120.0]","[76.0, 86.0, 62.0]","[16.0, nan, nan]","[nan, nan, nan]",...,[2 mg],[nan],"[[C3120220-1285-4FE1-A289-5C327786AAFA, 4879D0...",[5A5D58CC-8874-4DCB-8E38-ED2B89FAC111],"([[C3120220-1285-4FE1-A289-5C327786AAFA, 4879D...",[[Hypertrophy (Benign) of prostate ...,"[[0, 0], [0, 0, 0], [0, 0]]","[[600.0, 272.2], [600.0, 909.5, 272.2], [600.0...","[[0, 0], [0, 0, 0], [0, 0]]","[[nan, nan], [nan, nan, nan], [nan, nan]]"
1,"[CC8E82FF-2588-4D24-878A-05D0A7A82C68, 0BF992A...",0005D9BD-0247-4F02-B7EE-7C1B44825FA1,"[2009, 2010, 0, 0, 0, 0, 2009, 2010, 0, 2010, ...","[nan, 65.0, nan, 65.0, 65.0, 65.0, nan, nan, 6...","[0.0, 135.0, 0.0, 137.0, 137.0, 137.0, 0.0, 0....","[0.0, 22.463, 0.0, 22.796, 22.796, 22.796, 0.0...","[135.0, 130.0, nan, 128.0, 140.0, 133.0, 125.0...","[75.0, 80.0, nan, 60.0, 59.0, 73.0, 78.0, 82.0...","[nan, nan, nan, nan, nan, nan, 15.0, nan, nan,...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",...,[40 mg],[nan],"[[849004E3-E759-4F6E-ABC3-040DBCF510BF, 24852E...",[490BBDBE-DBA1-4CCC-A18C-51D00FC13837],"([[849004E3-E759-4F6E-ABC3-040DBCF510BF, 24852...","[[Mixed hyperlipidemia, Other psoriasis and si...","[[0, 0, 0], [0], [0, 0], [0, 0, 0], [0, 0, 0, 0]]","[[272.2, 696.1, 789.06], [696.1], [696.1, 272....","[[2009, 0, 2009], [0], [0, 2009], [0, 0, 2009]...","[[nan, nan, nan], [nan], [nan, nan], [nan, nan..."
2,"[582D3C53-0B68-4B68-96FA-0D5664D32392, B0D082A...",000B4862-7CE7-4EC5-8043-A97FCD74BD78,"[2012, 2012, 2011, 2011, 2012, 2012, 2012, 201...","[65.0, nan, nan, nan, nan, nan, nan, nan, nan,...","[100.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...","[16.639, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....","[100.0, 80.0, 83.0, nan, nan, nan, nan, nan, 1...","[60.0, 60.0, 56.0, nan, nan, nan, nan, nan, 60...","[18.0, 14.0, 14.0, nan, nan, nan, nan, nan, 20...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",...,"[10 mg/24 hr, 250 mcg (0.25 mg)]","[nan, nan]","[[0701D1C9-91CA-465D-9F7A-A9E7E8F5AE11, 79AC8C...","[EE1410C4-FF22-4BE3-9585-45E354C90341, EE1410C...","([[0701D1C9-91CA-465D-9F7A-A9E7E8F5AE11, 79AC8...","[[Impacted cerumen, Urinary incontinence, unsp...","[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0...","[[380.4, 788.30, 781.2, 458.9, 428.0, 783.0, 4...","[[0, 2012, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0],...","[[nan, nan, nan, nan, nan, nan, nan], [nan, na..."
3,"[DD03810C-ADF4-4B0D-B43D-01F9FCE71F55, 0393555...",00110ABC-DAB9-49E3-A1C8-88BBF8D58109,"[2012, 2011, 2012, 2011, 2009, 2010, 2012, 201...","[nan, nan, nan, nan, 71.0, nan, nan, nan, nan,...","[0.0, 0.0, 0.0, 0.0, 215.0, 0.0, 0.0, 0.0, 0.0...","[0.0, 0.0, 0.0, 0.0, 29.983, 0.0, 0.0, 0.0, 0....","[nan, nan, nan, nan, 130.0, nan, nan, nan, nan...","[nan, nan, nan, nan, 80.0, nan, nan, nan, nan,...","[nan, nan, nan, nan, 12.0, nan, nan, nan, nan,...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",...,"[-, Multiple Vitamins, 500 mg, 81 mg]","[nan, nan, nan, nan]","[[FB912E60-3DB4-42D9-A17D-B1E9171C0DB2], [D9BE...","[62C7E69B-EB1E-4711-A6D7-7AB644BEB1F9, 62C7E69...","([[FB912E60-3DB4-42D9-A17D-B1E9171C0DB2], [D9B...","[[Sebaceous cyst], [Other enthesopathy of ankl...","[[0], [0], [0], [0, 0, 0], [0], [0], [0], [0],...","[[706.2], [726.79], [706.2], [455, 272.1, V70]...","[[0], [0], [0], [0, 0, 0], [0], [0], [0], [0],...","[[nan], [nan], [nan], [nan, nan, nan], [nan], ..."
4,"[FE53B41D-1653-4090-B6D3-5B5E5BEA87FA, 361BD24...",002667F4-B9A3-4DE2-875F-8034DD637865,"[2010, 2010, 2010, 2010, 2010, 2010]","[59.0, nan, 59.0, 59.0, 59.0, 59.0]","[109.0, 0.0, 96.0, 100.0, 107.0, 99.0]","[22.013, 0.0, 19.388, 20.195, 21.609, 19.993]",

In [8]:
# Flattens the diagnosis array so that it's not nested
def flatten_icd9(codes):
    diags = []
    if len(codes) == 0:
        return
    
    for i in codes:
        for j in i:
            if len(j) != 0: diags.append(j)
                
    return diags

combined_data['ICD9CodeFlattened'] = combined_data.ICD9Code.apply(flatten_icd9)
combined_data.ICD9CodeFlattened.head()

0    [600.0, 272.2, 600.0, 909.5, 272.2, 600.0, 272.2]
1    [272.2, 696.1, 789.06, 696.1, 696.1, 272.2, 69...
2    [380.4, 788.30, 781.2, 458.9, 428.0, 783.0, 49...
3    [706.2, 726.79, 706.2, 455, 272.1, V70, 706.2,...
4    [240, 242.8, 242.8, 240, 242.8, 245.9, 240, 24...
Name: ICD9CodeFlattened, dtype: object

In [9]:
import re

# Categorizes the ICD9 codes into larger bins
def categorize_icd9(codes):
    bins = []

    if codes == None:
        return None
    
    for i in codes:
        # GROUP 1: infectious and parasitic diseases
        if bool(re.match('^00[0-9]', i)):
            bins.append("intestinal_infectious_diseases")
        elif bool(re.match('^01[0-8]', i)):
            bins.append("tuberculosis")
        elif bool(re.match('^02[0-7]', i)):
            bins.append("zoonotic_bacterial_diseases")
        elif bool(re.match('^03[0-9]|04[0-1]', i)):
            bins.append("other_bacterial_diseases")
        elif bool(re.match('^04[2-4]', i)):
            bins.append("hiv")
        elif bool(re.match('^05[0-9]', i)):
            bins.append("viral_diseases_exanthem")
        elif bool(re.match('^06[0-6]', i)):
            bins.append("arthropod_viral_diseases")
        elif bool(re.match('^07[0-9]', i)):
            bins.append("chlamydiae")
        elif bool(re.match('^08[0-8]', i)):
            bins.append("rickettsioses")
        elif bool(re.match('^09[0-9]', i)):
            bins.append("syphilis_venereal")
        elif bool(re.match('^10[0-4]', i)):
            bins.append("spirochetal_diseases")
        elif bool(re.match('^11[0-8]', i)):
            bins.append("mycoses")
        elif bool(re.match('^12[0-9]', i)):
            bins.append("helminthiases")
        elif bool(re.match('^13[0-6]', i)):
            bins.append("other_infectious_parasitic_diseases")
        elif bool(re.match('^13[7-9]', i)):
            bins.append("late_infectious_parasitic_diseases")
            
        # GROUP 2: neoplasms
        elif bool(re.match('^14[0-9]', i)):
            bins.append("oral_neoplasm")
        elif bool(re.match('^15[0-9]', i)):
            bins.append("digestive_neoplasm")
        elif bool(re.match('^16[0-5]', i)):
            bins.append("respiratory_neoplasm")
        elif bool(re.match('^17[0-5]', i)):
            bins.append("bone_neoplasm")
        elif bool(re.match('^176', i)):
            bins.append("kaposi_sarcoma")
        elif bool(re.match('^179|18[0-9]', i)):
            bins.append("genitourinary_neoplasm")
        elif bool(re.match('^19[0-9]', i)):
            bins.append("other_neoplasm")
        elif bool(re.match('^20[0-8]', i)):
            bins.append("lymphatic_neoplasm")
        elif bool(re.match('^2[1-2][0-9]', i)):
            bins.append("benign_neoplasm")
        elif bool(re.match('^23[0-4]', i)):
            bins.append("carcinoma_in_situ")
        elif bool(re.match('^23[5-8]', i)):
            bins.append("uncertain_neoplasm")
        elif bool(re.match('^239', i)):
            bins.append("unspecified_neoplasm")

        # GROUP 3: endocrine, nutritional, and metabolic diseases
        elif bool(re.match('^24[0-6]', i)):
            bins.append("thyroid_diseases")
        elif bool(re.match('^249', i)):
            bins.append("secondary_diabetes")
        elif bool(re.match('^250', i)):
            bins.append("diabetes")
        elif bool(re.match('^25[1-9]', i)):
            bins.append("other_endocrine_diseases")
        elif bool(re.match('^26[0-9]', i)):
            bins.append("nutritional_deficiencies")
        elif bool(re.match('^27[0-9]', i)):
            bins.append("other_metabolic_immunity_disorders")

        # GROUP 4: diseases of the blood, blood-forming organs
        elif bool(re.match('^28[0-5]', i)):
            bins.append("anemia")
        elif bool(re.match('^28[6-7]', i)):
            bins.append("coagulation_hemorrhagic")
        elif bool(re.match('^28[8-9]', i)):
            bins.append("other_blood")

        # GROUP 5: mental disorders
        elif bool(re.match('^29[0-9]', i)):
            bins.append("psychosis")
        elif bool(re.match('^300', i)):
            bins.append("neurotic_disorders")
        elif bool(re.match('^301', i)):
            bins.append("personality_disorders")
        elif bool(re.match('^302', i)):
            bins.append("psychosexual_disorders")
        elif bool(re.match('^30[3-5]', i)):
            bins.append("psychoactive_substance")
        elif bool(re.match('^30[6-9]|31[0-1]', i)):
            bins.append("other_mental_disorders")
        elif bool(re.match('^31[2-6]', i)):
            bins.append("child_mental_disorders")
        elif bool(re.match('^31[7-9]', i)):
            bins.append("mental_retardation")

        # GROUP 6a: diseases of the nervous system
        elif bool(re.match('^3[2-5][0-9]', i)):
            bins.append("inflammatory_central_nervous")
        elif bool(re.match('^33[0-7]', i)):
            bins.append("hereditary_degenerative_central_nervous")
        elif bool(re.match('^338', i)):
            bins.append("pain")
        elif bool(re.match('^339', i)):
            bins.append("headache")
        elif bool(re.match('^34[0-9]', i)):
            bins.append("other_central_nervous")
        elif bool(re.match('^35[0-9]', i)):
            bins.append("peripheral_nervous_diseases")

        # GROUP 6b: diseases of the sense organs
        elif bool(re.match('^3[6-7][0-9]', i)):
            bins.append("eye_adnexa_disorders")
        elif bool(re.match('^38[0-9]', i)):
            bins.append("ear_diseases")
            
        # GROUP 7: diseases of the circulatory system
        elif bool(re.match('^39[0-2]', i)):
            bins.append("acute_rheumatic_fever")
        elif bool(re.match('^39[3-8]', i)):
            bins.append("chronic_rheumatic_diseases")
        elif bool(re.match('^40[1-5]', i)):
            bins.append("hypertension")
        elif bool(re.match('^41[0-4]', i)):
            bins.append("ischemic_heart_disease")
        elif bool(re.match('^41[5-7]', i)):
            bins.append("pulmonary_circulation_diseases")
        elif bool(re.match('^42[0-9]', i)):
            bins.append("other_heart_disease")
        elif bool(re.match('^43[0-8]', i)):
            bins.append("cerebrovascular_disease")
        elif bool(re.match('^44[0-8]', i)):
            bins.append("arteries_diseases")
        elif bool(re.match('^45[1-9]', i)):
            bins.append("vein_lymphatics_diseases")

        # GROUP 8: diseases of the respiratory system
        elif bool(re.match('^46[0-6]', i)):
            bins.append("acute_respiratory_infections")
        elif bool(re.match('^47[0-8]', i)):
            bins.append("other_upper_respiratory_diseases")
        elif bool(re.match('^48[0-8]', i)):
            bins.append("pneumonia_influenza")
        elif bool(re.match('^49[0-6]', i)):
            bins.append("obstructive_pulmonary_disease")
        elif bool(re.match('^50[0-8]', i)):
            bins.append("pneumoconioses")
        elif bool(re.match('^51[0-9]', i)):
            bins.append("other_respiratory_diseases")

        # GROUP 9: diseases of the digesetive system
        elif bool(re.match('^52[0-9]', i)):
            bins.append("oral_diseases")
        elif bool(re.match('^53[0-7]', i)):
            bins.append("esophagus_diseases")
        elif bool(re.match('^54[0-3]', i)):
            bins.append("appendicitis")
        elif bool(re.match('^55[0-3]', i)):
            bins.append("abdominal_hernia")
        elif bool(re.match('^55[5-8]', i)):
            bins.append("enteritis_colitis")
        elif bool(re.match('^56[0-9]', i)):
            bins.append("other_intestines_peritoneum_diseases")
        elif bool(re.match('^57[0-9]', i)):
            bins.append("other_digestive_diseases")

        # GROUP 10: diseases of the genitourinary system
        elif bool(re.match('^58[0-9]', i)):
            bins.append("nephritis")
        elif bool(re.match('^59[0-9]', i)):
            bins.append("other_urinary_diseases")
        elif bool(re.match('^60[0-8]', i)):
            bins.append("male_genital_diseases")
        elif bool(re.match('^61[7-9]|62[0-9]', i)):
            bins.append("female_genital_diseases")

        # GROUP 11: complications of pregnancy, childbirth, and the puerperium
        elif bool(re.match('^63[0-3]', i)):
            bins.append("ectopic_molar_pregnancy")
        elif bool(re.match('^63[4-9]', i)):
            bins.append("other_preg_abortive_outcome")
        elif bool(re.match('^64[0-9]', i)):
            bins.append("complications_preg")
        elif bool(re.match('^65[0-9]', i)):
            bins.append("normal_delivery")
        elif bool(re.match('^66[0-9]', i)):
            bins.append("complications_labor")
        elif bool(re.match('^67[0-6]', i)):
            bins.append("complications_puerperium")
        elif bool(re.match('^67[8-9]', i)):
            bins.append("other_maternal_fetal_complications")

        # GROUP 12: diseases of the skin and subcutaneous tissue
        elif bool(re.match('^68[0-6]', i)):
            bins.append("infections_skin")
        elif bool(re.match('^69[0-8]', i)):
            bins.append("other_inflammatory_skin")
        elif bool(re.match('^70[0-9]', i)):
            bins.append("other_diseases_skin")

        # GROUP 13: diseases of the musculoskeletal system and connective tissue
        elif bool(re.match('^71[0-9]', i)):
            bins.append("arthropathies")
        elif bool(re.match('^72[0-4]', i)):
            bins.append("dorsopathies")
        elif bool(re.match('^72[5-9]', i)):
            bins.append("rheumatism")
        elif bool(re.match('^73[0-9]', i)):
            bins.append("osteopathies")

        # GROUP 14: congenital anomalies
        elif bool(re.match('^74[0-2]', i)):
            bins.append("congenital_nervous")
        elif bool(re.match('^74[3-4]', i)):
            bins.append("congenital_face")
        elif bool(re.match('^74[5-7]', i)):
            bins.append("congenital_circulatorhy")
        elif bool(re.match('^748', i)):
            bins.append("congenital_respiratory")
        elif bool(re.match('^749|75[0-1]', i)):
            bins.append("congenital_digestive")
        elif bool(re.match('^752', i)):
            bins.append("congenital_genital")
        elif bool(re.match('^753', i)):
            bins.append("congenital_urinary")
        elif bool(re.match('^75[4-6]', i)):
            bins.append("congenital_musculoskeletal")
        elif bool(re.match('^757', i)):
            bins.append("congenital_integument")
        elif bool(re.match('^758', i)):
            bins.append("congenital_chromosomal")
        elif bool(re.match('^759', i)):
            bins.append("congenital_other")

        # GROUP 15: certain conditions originatiing in the perinatal period
        elif bool(re.match('^76[0-3]', i)):
            bins.append("maternal_perinatal")
        elif bool(re.match('^76[4-9]|77[0-9]', i)):
            bins.append("other_perinatal")
            
        # GROUP 16: symptoms, signs, and ill-defined conditions
        elif bool(re.match('^78[0-9]', i)):
            bins.append("symptoms")
        elif bool(re.match('^79[0-6]', i)):
            bins.append("nonspecific_abnormal_findings")
        elif bool(re.match('^79[7-9]', i)):
            bins.append("illdefined_unknown_causes")

        # GROUP 17: injury and poisoning
        elif bool(re.match('^8[0-9][0-9]|9[0-9][0-9]', i)):
            bins.append("injury_poisoning")
            
        # GROUP E: external causes of injury
        elif bool(re.match('^e|E', i)):
            bins.append("external_injury")
            
        # GROUP V: supplementary classification
        elif bool(re.match('^v|V', i)):
            bins.append("supplementary_classification")

    return bins

combined_data['ICD9Binned'] = combined_data.ICD9CodeFlattened.apply(categorize_icd9)
combined_data.ICD9Binned.head()

KeyboardInterrupt: 

In [ ]:
icd9_data = pd.get_dummies(combined_data.ICD9Binned.apply(pd.Series).stack()).sum(level=0).apply(np.sign)
icd9_data['PatientGuid'] = combined_data['PatientGuid']
icd9_data.head()

In [ ]:
icd9_data.to_csv('combined_data.csv')

In [ ]:
icd9_data[icd9_data.diabetes == 1]